In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv('data_raw.csv')

In [3]:
df.head()

,Smiles,TC+DA,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,[H]C1=CC=C([S+](C(F)(F)F)C2=C3C=CC=C2)C3=C1,42.7,13.647975,11.749784,0,1,21.736982,2.562927,4.918280,21.736982,...,10.052726,65.315437,253.029332,10.121173,439,29,96,117,5.618056,3.555556
1,FC([S+]1C2=CC=C(OC)C=C2C3=C1C=CC=C3)(F)F,45.5,15.062188,12.816932,0,1,24.176730,2.571754,4.937593,24.176730,...,10.141441,67.786962,283.039897,9.759996,617,33,106,129,6.729167,4.055556
2,FC([S+]1C2=CC=C(C)C=C2C3=C1C=CC=C3)(F)F,44.1,14.464471,12.359148,0,1,22.700235,2.570047,4.932580,22.700235,...,10.115408,66.617442,267.044982,9.537321,519,31,102,124,6.479167,3.722222
3,FC1=CC=C([S+](C(F)(F)F)C2=C3C=CC=C2)C3=C1,42.1,14.464471,12.359148,0,1,22.700235,2.570047,4.932580,22.700235,...,10.115408,66.617442,271.019910,10.840796,519,31,102,124,6.479167,3.722222
4,ClC1=CC=C([S+](C(F)(F)F)C2=C3C=CC=C2)C3=C1,41.0,14.464471,12.359148,0,1,22.700235,2.570047,4.932580,22.700235,...,10.115408,66.617442,286.990360,11.479614,519,31,102,124,6.479167,3.722222


In [4]:
final_df = df.drop(['Smiles', 'TC+DA'], axis=1)

In [5]:
# prompt: print columns containing string values
bool_cols = []
num_cols = []
for col in final_df.columns:
  if final_df[col].dtype == 'object':
    pass
  elif final_df[col].dtype =='bool':
      bool_cols.append(col)
  else:
      num_cols.append(col)

In [6]:
final_df.isnull().sum().sum()

np.int64(0)

In [7]:
bool_cols

['Lipinski', 'GhoseFilter']

In [8]:
len(num_cols)

1502

In [9]:
final_df_3d = final_df[num_cols + bool_cols]

In [10]:
final_df_3d.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,Lipinski,GhoseFilter
0,13.647975,11.749784,0,1,21.736982,2.562927,4.918280,21.736982,1.278646,3.799815,...,253.029332,10.121173,439,29,96,117,5.618056,3.555556,False,True
1,15.062188,12.816932,0,1,24.176730,2.571754,4.937593,24.176730,1.272459,3.901478,...,283.039897,9.759996,617,33,106,129,6.729167,4.055556,False,True
2,14.464471,12.359148,0,1,22.700235,2.570047,4.932580,22.700235,1.261124,3.854099,...,267.044982,9.537321,519,31,102,124,6.479167,3.722222,False,True
3,14.464471,12.359148,0,1,22.700235,2.570047,4.932580,22.700235,1.261124,3.854099,...,271.019910,10.840796,519,31,102,124,6.479167,3.722222,False,True
4,14.464471,12.359148,0,1,22.700235,2.570047,4.932580,22.700235,1.261124,3.854099,...,286.990360,11.479614,519,31,102,124,6.479167,3.722222,False,False


In [11]:
def remove_constant_values(data):
    return [e for e in data.columns if data[e].nunique() == 1]

drop_col = remove_constant_values(final_df_3d)
#drop_col

new_df_columns = [e for e in final_df_3d.columns if e not in drop_col]
new_df = final_df_3d[new_df_columns]
new_df.head(2)

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,Lipinski,GhoseFilter
0,13.647975,11.749784,0,1,21.736982,2.562927,4.918280,21.736982,1.278646,3.799815,...,253.029332,10.121173,439,29,96,117,5.618056,3.555556,False,True
1,15.062188,12.816932,0,1,24.176730,2.571754,4.937593,24.176730,1.272459,3.901478,...,283.039897,9.759996,617,33,106,129,6.729167,4.055556,False,True


In [12]:
# To calclulate  Correlation and remove highly  correlated columns
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [13]:
# Dropping highly correlated Features
corr_features = correlation(new_df, 0.80)
print("No. of features to drop : ",len(set(corr_features)))

new_df.drop(corr_features,axis=1,inplace=True)

No. of features to drop :  1033


/tmp/ipython-input-2887508640.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(corr_features,axis=1,inplace=True)


In [14]:
new_df.shape

(99, 249)

In [15]:
new_df.head(2)

,ABC,nAcid,nBase,SpMax_A,SpMAD_A,nAromAtom,nSpiro,nBridgehead,nHetero,nN,...,JGI3,JGI4,JGI6,JGI7,JGI8,JGI9,JGI10,TopoShapeIndex,Lipinski,GhoseFilter
0,13.647975,0,1,2.562927,1.278646,12,0,0,4,0,...,0.083333,0.047222,0.029383,0.000000,0.000000,0.0,0.0,0.5,False,True
1,15.062188,0,1,2.571754,1.272459,12,0,0,5,0,...,0.075126,0.050926,0.031949,0.019593,0.005102,0.0,0.0,1.0,False,True


In [16]:
new_df['Lipinski'] = new_df["Lipinski"].astype(int)
new_df['GhoseFilter'] = new_df["GhoseFilter"].astype(int)

/tmp/ipython-input-1775495275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Lipinski'] = new_df["Lipinski"].astype(int)
/tmp/ipython-input-1775495275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['GhoseFilter'] = new_df["GhoseFilter"].astype(int)


In [17]:
new_df.head(2)

,ABC,nAcid,nBase,SpMax_A,SpMAD_A,nAromAtom,nSpiro,nBridgehead,nHetero,nN,...,JGI3,JGI4,JGI6,JGI7,JGI8,JGI9,JGI10,TopoShapeIndex,Lipinski,GhoseFilter
0,13.647975,0,1,2.562927,1.278646,12,0,0,4,0,...,0.083333,0.047222,0.029383,0.000000,0.000000,0.0,0.0,0.5,0,1
1,15.062188,0,1,2.571754,1.272459,12,0,0,5,0,...,0.075126,0.050926,0.031949,0.019593,0.005102,0.0,0.0,1.0,0,1


In [18]:
new_df.to_csv('data.csv', index=False)